In [ ]:
##################################### I #######################################
import polars as pl

# Считаем датасет из CSV файла
df = pl.read_csv('https://drive.google.com/uc?id=1QoPbjyl7xsMjd6PWuM_1asJDfid3ObkF&export=download')

# 1. Основная информация о датасете
print("Типы данных по столбцам:")
print(df.dtypes)

print("\n-Количество пропусков по столбцам:")
print(df.null_count())

print("\n-Статистическое описание (describe):")
print(df.describe())

print("\n-Средние значения по числовым столбцам:")
print(df.mean())

# 2. Количество пассажиров каждого класса (Pclass)
pclass_counts = df.get_column("Pclass").value_counts(sort=True, name="count")
print("\n-Количество пассажиров по классам (Pclass):")
print(pclass_counts)

# 3. Количество выживших мужчин и женщин
survivors_gender = (
    df.filter(pl.col("Survived") == 1)
      .group_by("Sex")
      .agg(pl.col("Survived").sum().alias("Survived_Count"))
)
print("\n-Количество выживших мужчин и женщин:")
print(survivors_gender)

# 4. Пассажиры с возрастом больше 44 лет
old_passengers = df.filter(pl.col("Age") > 44)
print("\n-Пассажиры с возрастом больше 44 лет:")
print(old_passengers)

#################################### II #######################################

# Считаем датасет train.csv с помощью pandas из Google Drive
# Загрузка датасета

import pandas as pd
import bottleneck as bn

url = 'https://drive.google.com/uc?id=1QoPbjyl7xsMjd6PWuM_1asJDfid3ObkF&export=download'
df = pd.read_csv(url)

# 1. Посчитаем средний возраст и стандартное отклонение, используя bottleneck:

mean_age_bn = bn.nanmean(df['Age'])
std_age_bn = bn.nanstd(df['Age'])

print(f"Средний возраст (bottleneck): {mean_age_bn:.2f}")
print(f"Стандартное отклонение возраста (bottleneck): {std_age_bn:.2f}")

# 2. Для каждого пассажира умножим значение столбца Fare на 1.3 и сохраним результаты в новом столбце Fare_new.
# 2.0 Создание нового столбца Fare_new с использованием .itertuples()

fare_new_itertuples = []
for row in df.itertuples():
    fare_new_itertuples.append(row.Fare * 1.3)
df['Fare_new_itertuples'] = fare_new_itertuples
print("Столбец Fare_new_itertuples создан.")

# 2.1. Создание нового столбца Fare_new с использованием .apply()

df['Fare_new_apply'] = df['Fare'].apply(lambda x: x * 1.3)
print("Столбец Fare_new_apply создан.")

# Теперь посмотрим на первые несколько строк обновленного датасета

print("\n-Первые несколько строк обновленного датасета:")
print(df.head())

################################### III #######################################
# Датасет Housing.csv можно считать напрямую из Google Drive с помощью pandas

import pandas as pd

url = 'https://drive.google.com/uc?id=11Wxpzg4YOeTa-OibHqGKt1Dr9xZxdtdj&export=download'
df = pd.read_csv(url)

# Определение оптимальных типов для каждого столбца:

optimized_types = {}

for col in df.columns:
    if pd.api.types.is_float_dtype(df[col]):
        # Приводим float64 к float32
        optimized_types[col] = 'float32'
    elif pd.api.types.is_integer_dtype(df[col]):
        min_val = df[col].min()
        max_val = df[col].max()
        if min_val >= 0 and max_val < 256:
            optimized_types[col] = 'int8'
        elif min_val >= -32768 and max_val < 32768:
            optimized_types[col] = 'int16'
        else:
            optimized_types[col] = 'int32'
    else:
        # Для строковых столбцов с небольшим числом уникальных значений - category
        if df[col].nunique() / len(df) < 0.5:
            optimized_types[col] = 'category'
        else:
            optimized_types[col] = df[col].dtype

################################################ Комментарии по выбору типов #######################################
# В данном датасете столбцы с бинарными признаками (например, mainroad, guestroom, basement и др.) лучше перевести в категориальный тип.
# Количество комнат, этажей и парковочных мест - целочисленные с небольшим диапазоном, подходят int8 или int16.
# Цена (price) - целочисленный, но с большим диапазоном, лучше int32.
# Площадь (area) - целочисленная, можно int16 (если значения не выходят за пределы).
#####################################################################################################################

# 3. Применение оптимизированных типов и сравнение памяти.

# Потребление памяти до оптимизации.
memory_before = df.memory_usage(deep=True).sum() / 1024**2  # в мегабайтах
print(f'Потребление памяти до оптимизации: {memory_before:.4f} МБ')

# Применение новых типов:
for col, dtype in optimized_types.items():
    df[col] = df[col].astype(dtype)

# Потребление памяти после оптимизации:
memory_after = df.memory_usage(deep=True).sum() / 1024**2  # в мегабайтах
print(f'Потребление памяти после оптимизации: {memory_after:.4f} МБ')

Типы данных по столбцам:
[Int64, Int64, Int64, String, String, Float64, Int64, Int64, String, Float64, String, String]

-Количество пропусков по столбцам:
shape: (1, 12)
┌─────────────┬──────────┬────────┬──────┬───┬────────┬──────┬───────┬──────────┐
│ PassengerId ┆ Survived ┆ Pclass ┆ Name ┆ … ┆ Ticket ┆ Fare ┆ Cabin ┆ Embarked │
│ ---         ┆ ---      ┆ ---    ┆ ---  ┆   ┆ ---    ┆ ---  ┆ ---   ┆ ---      │
│ u32         ┆ u32      ┆ u32    ┆ u32  ┆   ┆ u32    ┆ u32  ┆ u32   ┆ u32      │
╞═════════════╪══════════╪════════╪══════╪═══╪════════╪══════╪═══════╪══════════╡
│ 0           ┆ 0        ┆ 0      ┆ 0    ┆ … ┆ 0      ┆ 0    ┆ 687   ┆ 2        │
└─────────────┴──────────┴────────┴──────┴───┴────────┴──────┴───────┴──────────┘

-Статистическое описание (describe):
shape: (9, 13)
┌────────────┬─────────────┬──────────┬──────────┬───┬───────────┬───────────┬───────┬──────────┐
│ statistic  ┆ PassengerId ┆ Survived ┆ Pclass   ┆ … ┆ Ticket    ┆ Fare      ┆ Cabin ┆ Embarked │
│ ---  